Lista 3

In [146]:
import numpy as np
from scipy.linalg import solve

In [164]:
A = np.array([
    [1, 1/2, 1/3, 1/4, 1/5],
    [1/2, 1/3, 1/4, 1/5, 1/6],
    [1/3, 1/4, 1/5, 1/6, 1/7],
    [1/4, 1/5, 1/6, 1/7, 1/8],
    [1/5, 1/6, 1/7, 1/8, 1/9]
])

b = np.array([5,4,3,2,1])

augmented_matrix = np.column_stack((A, b))

In [173]:
def lu(mat):
    n = len(mat)
    L, U = np.zeros((n,n)), np.zeros((n,n))

    for i in range(n):
        for k in range(i, n):
            sum_ = 0
            for j in range(i):
                sum_ += (L[i][j] * U[j][k])
            U[i][k] = mat[i][k] - sum_

        for k in range(i, n):
            if (i == k):
                L[i][i] = 1 
            else:
                sum_ = 0
                for j in range(i):
                    sum_ += (L[k][j] * U[j][i])
                L[k][i] = (mat[k][i] - sum_) / U[i][i]
    return L, U


In [174]:
L,U = lu(A)

In [186]:
print(L,U)

[[1.         0.         0.         0.         0.        ]
 [0.5        1.         0.         0.         0.        ]
 [0.33333333 1.         1.         0.         0.        ]
 [0.25       0.9        1.5        1.         0.        ]
 [0.2        0.8        1.71428571 2.         1.        ]] [[1.00000000e+00 5.00000000e-01 3.33333333e-01 2.50000000e-01
  2.00000000e-01]
 [0.00000000e+00 8.33333333e-02 8.33333333e-02 7.50000000e-02
  6.66666667e-02]
 [0.00000000e+00 0.00000000e+00 5.55555556e-03 8.33333333e-03
  9.52380952e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 3.57142857e-04
  7.14285714e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.26757370e-05]]


In [187]:
y = solve(L, b)

In [188]:
x = solve(U, y)

In [202]:
print(x)

[   -95.           2159.99999999 -10709.99999998  17919.99999997
  -9449.99999999]


In [190]:
Ax = np.dot(A, x)

In [191]:
r = b - Ax

In [192]:
print(r)

[ 1.42108547e-14 -3.90798505e-13 -3.34843264e-13  4.54747351e-13
  1.94400052e-13]


In [207]:
norma_r = np.linalg.norm(r,np.inf)
norma_Ax = np.linalg.norm(Ax, np.inf)

In [208]:
print(norma_r, norma_Ax)

4.547473508864641e-13 4.999999999999986


In [197]:
def dokladnosc_maszynowa(x_0):
    x = x_0
    while x + 1 != 1:
        x /= 2
    return x

In [198]:
u = dokladnosc_maszynowa(1)

In [199]:
delta_x = solve(A, r)

In [206]:
def iteracyjne_poprawianie_rozwiazan(A, b):
    L,U = lu(A) #rozkład LU
    y = solve(L, b) #rozwiązujemy za pomocą LU
    x = solve(U, y)
    Ax = np.dot(A, x)
    r = b - Ax #reszty
    u = dokladnosc_maszynowa(1)
    while np.linalg.norm(r,np.inf) < np.linalg.norm(Ax, np.inf) * u:
        delta_x = solve(A, r)
        x += delta_x
        r = b - np.dot(A, x)
    return x

In [205]:
print(iteracyjne_poprawianie_rozwiazan(A, b))

[   -95.           2159.99999999 -10709.99999998  17919.99999997
  -9449.99999999]
